*   실행환경: 구글 코랩
*   작성일: 230930
*   작성자: 이혜원



### 라이브러리 설치, 불러오기

In [ ]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
#라이브러리 불러오기
import pandas as pd
import re
from konlpy.tag import Mecab

### 데이터 불러오기

In [ ]:
song_df = pd.read_csv('가사.csv')
rank_df = pd.read_csv('순위.csv')
info_df = pd.read_csv('앨범정보.csv')

In [ ]:
print('song_df.shape:', song_df.shape)
print('rank_df.shape:', rank_df.shape)
print('info_df.shape:', info_df.shape)

song_df.shape: (1107, 2)
rank_df.shape: (1300, 8)
info_df.shape: (1010, 6)


### 사용자 정의 함수 만들기

함수




*   preprocess(sentences) : 가사 전처리
*   tokenize(sentences) : 형태소 분석기 mecab을 사용한 토큰화 작업 (다소 시간 소요)
*   find_rank_song(song_id) : song_id로 rank_df 행 찾기
*   find_rank_album(album_id) : album_id로 rank_df 행 찾기
*   find_info_id(album_id) : album_id로 info_df 행 찾기
*   find_genre_song(song_id) : song_id를 넣으면 genre를 반환
*   find_genres_songs(song_id_list) : song_id_list를 넣으면 genre_list를 반환
*   make_unique_lst(df_column) : unique한 list 구하기
*   make_word_to_idx(unique_lst) : 리턴 -> dict {"A" :1, "B" :2, ... }  형식
*   ⭐make_idx_list(column_lst) : 2차원 리스트의 단어를 숫자와 매핑


In [ ]:
# 가사 전처리
def preprocess(sentences):
    lst = []
    for sentence in sentences:
        # \n과 \t를 제거한다
        sentence = re.sub('\\t', '', sentence)
        sentence = re.sub('\\n', '', sentence)
        # 한국어, 공백 빼고 제거하기
        sentence = re.sub('[^ㄱ-ㅎ가-힣\s]+', '', sentence)
        # 문장 양옆의 띄어쓰기를 지운다
        sentence = sentence.strip()

        lst.append(sentence)
    return lst

In [ ]:
# 형태소 분석기 mecab을 사용한 토큰화 작업 (다소 시간 소요)
def tokenize(sentences):
    mecab = Mecab()
    tokenized_data = []
    for sentence in sentences:
        temp_X = mecab.morphs(sentence)
        temp_X = [word for word in temp_X]
        tokenized_data.append(temp_X)
    return tokenized_data

In [ ]:
# song_id로 rank_df 행 찾기
def find_rank_song(song_id):
    return rank_df[rank_df['song_id'] ==  song_id]

# album_id로 rank_df 행 찾기
def find_rank_album(album_id):
    return rank_df[rank_df['album_id'] ==  album_id]

# album_id로 info_df 행 찾기
def find_info_id(album_id):
    return info_df[info_df['AlbumID'] == album_id]

# song_id를 넣으면 genre를 반환
def find_genre_song(song_id):
    album_id = find_rank_song(song_id)['album_id'].values[0]
    genre = find_info_id(album_id)['Genre'].values[0]
    return genre

# song_id_list를 넣으면 genre_list를 반환
def find_genres_songs(song_id_list):
    result = []
    for song_id in song_id_list:
        genre = find_genre_song(song_id)
        result.append(genre)
    return result

In [ ]:
# unique한 장르 list 구하기
def make_unique_lst(df_column):
    unique_lst = df_column.unique()
    # print(len(unique_lst)) #89

    # 한 행의 장르가 '댄스, 랩/힙합, 록/메탈' 로 콤마로 여러개가 표시되어 있는 경우가 있음
    split_lst = [val.split(', ') for val in unique_lst]
    sum_lst = sum(split_lst, [])  # 리스트 내 리스트 합치기
    unique_lst = list(set(sum_lst))  # 중복 제거
    return unique_lst

In [ ]:
# 리턴 -> dict {"A" :1, "B" :2, ... }  형식
def make_word_to_idx(unique_lst):
    unique_lst = make_unique_lst(unique_lst)
    word_to_idx = {val: idx for idx, val in enumerate(unique_lst)}
    return word_to_idx

In [ ]:
# 2차원 리스트를 넣으면 단어 각각을 숫자와 매칭시키고 숫자로 바꾼다.

# input
# 숫자로 바꾸고 싶은 데이터 프레임 열

# return
# 단어를 숫자로 매핑한 2차원 배열, word_to_idx

# column_list: 데이터 프레임 열
def make_idx_list(column_lst):
    df_column_list = [tmp.split(", ") for tmp in column_lst]
    word_to_idx = make_word_to_idx(column_lst)

    new_column = []
    for column in df_column_list:
        tmp_lst = []
        for one_column in column:
            tmp_lst.append(word_to_idx[one_column])
        new_column.append(tmp_lst)

    return new_column, word_to_idx


### 전처리

#### 비어있는 행 확인

In [ ]:
info_df.isna().sum()

Unnamed: 0      0
AlbumID         0
Release_date    0
Genre           0
Publisher       0
Agency          0
dtype: int64

In [ ]:
song_df.isna().sum()

Song_ID    0
Lyrics     0
dtype: int64

In [ ]:
rank_df.isna().sum()

Unnamed: 0    0
year          0
rank          0
song_name     0
artist        0
likes         0
album_id      0
song_id       0
dtype: int64

#### song_df에 숫자로 매칭된 genre 열 생성

In [ ]:
song_df['genre'] = find_genres_songs(song_df['Song_ID'])

In [ ]:
song_df

,Song_ID,Lyrics,genre
0,34373632,어린 햇살 아래서뛰어놀곤 했었던가쁜 숨결굽이진 골목 지나길을 따라가보면같은 기억어른...,"국내드라마, 발라드"
1,8124417,꽃이 언제 피는지 그딴 게 뭐가 중요한데날씨가 언제 풀리는지 그딴 거 알면 뭐 할 ...,"인디음악, 포크/블루스"
2,2705410,늦게 다니지좀 마술은 멀리좀 해봐열살짜리 애처럼 말을 안듣니정말 웃음만 나와누가 누...,발라드
3,30205955,난 또 행복한 척 더 더 행복한 척난 또 행복한 척 더 더 행복한 척아무에게도 말하...,R&B/Soul
4,3590149,밥을 먹어도 혼자야구를 봐도 혼자거리를 걸어도 나 혼자혼자이기 싫은데또 방에 누워도...,발라드
...,...,...,...
1102,3713013,닫힌 내 가슴은 누구도 사랑할 수가 없다 그렇게 믿었는데 어느새 내 가슴이 열리...,댄스
1103,4020215,네가 떠나고 늘 슬프다 아무리 슬퍼해도 슬픔이 모자라 맘 놓고 아파할 곳을 찾아 숨...,랩/힙합
1104,2797558,오 내 맘이야 맘이야 맘이 맘이야 오 내 맘이야 맘이야 맘이 맘이야 Ma Ma Ma...,"댄스, 발라드"
1105,31873020,그대와 자주 가던 그 술집에혼자 널 생각하며 소주 한잔해그대가 좋아하던 김치찌개를가...,발라드


In [ ]:
song_df['genre'], genre_to_idx = make_idx_list(song_df['genre'])

In [ ]:
genre_to_idx

{'댄스': 0,
 '발라드': 1,
 'R&B/Soul': 2,
 '랩/힙합': 3,
 '성인가요/트로트': 4,
 '록/메탈': 5,
 '국내드라마': 6,
 '일렉트로니카': 7,
 '포크/블루스': 8,
 '크로스오버': 9,
 '애시드/퓨전/팝': 10,
 '클래식': 11,
 '재즈': 12,
 '인디음악': 13}

In [ ]:
song_df.tail(3)

,Song_ID,Lyrics,genre
1104,2797558,오 내 맘이야 맘이야 맘이 맘이야 오 내 맘이야 맘이야 맘이 맘이야 Ma Ma Ma...,"[0, 1]"
1105,31873020,그대와 자주 가던 그 술집에혼자 널 생각하며 소주 한잔해그대가 좋아하던 김치찌개를가...,[1]
1106,2705407,Push Push Baby Oh Push BabyPush Push baby Just...,"[0, 1]"


#### song_df에 가사(Lyrics) 전처리

In [ ]:
song_df['Lyrics'] = preprocess(song_df['Lyrics'])

In [ ]:
song_df.tail(3)

,Song_ID,Lyrics,genre
1104,2797558,오 내 맘이야 맘이야 맘이 맘이야 오 내 맘이야 맘이야 맘이 맘이야 ...,"[0, 1]"
1105,31873020,그대와 자주 가던 그 술집에혼자 널 생각하며 소주 한잔해그대가 좋아하던 김치찌개를가...,[1]
1106,2705407,맘을 받아줘 달콤한 말로 ...,"[0, 1]"


In [ ]:
song_df

,Song_ID,Lyrics,genre
0,34373632,어린 햇살 아래서뛰어놀곤 했었던가쁜 숨결굽이진 골목 지나길을 따라가보면같은 기억어른...,"[6, 1]"
1,8124417,꽃이 언제 피는지 그딴 게 뭐가 중요한데날씨가 언제 풀리는지 그딴 거 알면 뭐 할 ...,"[13, 8]"
2,2705410,늦게 다니지좀 마술은 멀리좀 해봐열살짜리 애처럼 말을 안듣니정말 웃음만 나와누가 누...,[1]
3,30205955,난 또 행복한 척 더 더 행복한 척난 또 행복한 척 더 더 행복한 척아무에게도 말하...,[2]
4,3590149,밥을 먹어도 혼자야구를 봐도 혼자거리를 걸어도 나 혼자혼자이기 싫은데또 방에 누워도...,[1]
...,...,...,...
1102,3713013,닫힌 내 가슴은 누구도 사랑할 수가 없다 그렇게 믿었는데 어느새 내 가슴이 열리...,[0]
1103,4020215,네가 떠나고 늘 슬프다 아무리 슬퍼해도 슬픔이 모자라 맘 놓고 아파할 곳을 찾아 숨...,[3]
1104,2797558,오 내 맘이야 맘이야 맘이 맘이야 오 내 맘이야 맘이야 맘이 맘이야 ...,"[0, 1]"
1105,31873020,그대와 자주 가던 그 술집에혼자 널 생각하며 소주 한잔해그대가 좋아하던 김치찌개를가...,[1]


In [ ]:
# 반복되는 가사를 어떻게 한번만 등장하게끔 만들 수 있을까?

In [ ]:
# 한 노래의 장르가 여러개일 경우 어떻게 처리하지?
# 예측할 때 하나라도 맞추면 맞았다고 해야하는지?
# 다중 레이블 분류 찾아보기

### 모델 돌려보기

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import datetime
from tqdm import tqdm

import csv
import os

import tensorflow as tf
import torch

# BERT 사용을 위함
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")

In [ ]:
# 훈련 데이터, 검증 데이터, 테스트 데이터를 로드합니다.

train = load_dataset("jeanlee/kmhas_korean_hate_speech", split="train")
validation = load_dataset("jeanlee/kmhas_korean_hate_speech", split="validation")
test = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

In [ ]:
train['label']

In [ ]:
train['text']

### 데이터 준비하기

#### 데이터셋 만들기 - 훈련 데이터, 테스트 데이터 나누기

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class SongDataset(Dataset):
    def __init__(self, x_data, y_data):
        # self.data = pd.DataFrame(data, columns=['lyrics', 'genre'])
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]

        return x, y

In [ ]:
# 사용할 데이터 : song_df
X = song_df['Lyrics']
Y = song_df['genre']

In [ ]:
# 학습데이터 테스트 데이터 나누기 - 80% 20%
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size = 0.2, random_state = 42
)

train_data, train_label, test_data, test_label

In [ ]:
train_data = X_train.reset_index(drop=True)
train_label = y_train.reset_index(drop=True)
test_data = X_test.reset_index(drop=True)
test_label = y_test.reset_index(drop=True)

In [ ]:
# train_dataset = SongDataset(train_data)
# test_dataset = SongDataset(test_data)
train_dataset = SongDataset(train_data, train_label)
test_dataset = SongDataset(test_data, test_label)

In [ ]:
train_dataset.__getitem__(3)

('어쩜 이렇게 하늘은 더 파란 건지 오늘따라 왜 바람은 또 완벽한지 그냥 모르는 척 하나 못들은 척 지워버린 척 딴 얘길 시작할까 아무 말 못하게 입맞출까 눈물이 차올라서 고갤 들어 흐르지 못하게 또 살짝 웃어 내게 왜 이러는지 무슨 말을 하는지 오늘 했던 모든 말 저 하늘 위로 한번도 못했던 말 울면서 할 줄은 나 몰랐던 말 나는요 오빠가 좋은걸 어떡해 새로 바뀐 내 머리가 별로였는지 입고 나왔던 옷이 실수였던 건지 아직 모르는 척 기억 안 나는 척 아무 일없던 것처럼 굴어볼까 그냥 나가자고 얘기할까 눈물이 차올라서 고갤 들어 흐르지 못하게 또 살짝 웃어 내게 왜 이러는지 무슨 말을 하는지 오늘 했던 모든 말 저 하늘 위로 한번도 못했던 말 울면서 할 줄은 나 몰랐던 말 나는요 오빠가 좋은걸 어떡해 이런 나를 보고 그런 슬픈 말은 하지 말아요 철없는 건지 조금 둔한 건지 믿을 수가 없는걸요 눈물은 나오는데 활짝 웃어 네 앞을 막고서 막 크게 웃어 내가 왜 이러는지 부끄럼도 없는지 자존심은 곱게 접어 하늘위로 한 번도 못했던 말 어쩌면 다신 못할 바로 그 말 나는요 오빠가 좋은걸 아이쿠 하나 둘                    이렇게 좋은 날',
 [1, 0])

#### 데이터셋 전처리

In [ ]:
train_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', train_data))
test_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', test_data))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
enc = MultiLabelBinarizer()
enc.fit([list(range(0, 14))])

def multi_label(label):
    enc_label = enc.transform(label)
    float_arr = np.vstack(enc_label[:]).astype(float)
    update_label = float_arr.tolist()
    return update_label

In [ ]:
genre_to_idx

{'댄스': 0,
 '발라드': 1,
 'R&B/Soul': 2,
 '랩/힙합': 3,
 '성인가요/트로트': 4,
 '록/메탈': 5,
 '국내드라마': 6,
 '일렉트로니카': 7,
 '포크/블루스': 8,
 '크로스오버': 9,
 '애시드/퓨전/팝': 10,
 '클래식': 11,
 '재즈': 12,
 '인디음악': 13}

In [ ]:
train_label

0       [1, 2]
1       [0, 1]
2      [2, 13]
3       [1, 0]
4       [0, 1]
        ...   
880     [1, 6]
881        [1]
882     [0, 1]
883     [0, 1]
884     [1, 6]
Name: genre, Length: 885, dtype: object

In [ ]:
train_labels = multi_label(train_label)
test_labels = multi_label(test_label)

In [ ]:
print(len(train_labels), len(train_labels[0]))
print(len(test_labels), len(test_labels[0]))

885 14
222 14


#### BERT 토크나이저를 이용한 전처리

In [ ]:
# 한국어 BERT 중 하나인 'klue/bert-base'를 사용.
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [ ]:
# train_data, train_label, test_data, test_label

In [ ]:
max([len(lyrics) for lyrics in song_df['Lyrics']])

1888

In [ ]:
tokenizer.tokenize('가나다')

['가나', '##다']

In [ ]:
MAX_LEN = 400

def data_to_tensor(sentences, labels):
    # 정수 인코딩
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
     # pad_sequence: 패딩, 주어진 최대 길이보다 짧으면 뒤에 0으로 채워준다
    input_ids = pad_sequences(list(input_ids), maxlen=MAX_LEN, dtype="long",
                              truncating = "post", padding="post")
    attention_masks = []

    for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)

    tensor_inputs = torch.tensor(input_ids)
    tensor_labels = torch.tensor(labels)
    tensor_masks = torch.tensor(attention_masks)

    return tensor_inputs, tensor_labels, tensor_masks

train_sentences, test_sentences, train_labels, test_labels

In [ ]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)


In [ ]:
train_labels.shape

torch.Size([885, 14])

In [ ]:
test_labels.shape

torch.Size([222, 14])

In [ ]:
batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
print('훈련 데이터의 크기:', len(train_labels))
print('테스트 데이터의 크기:', len(test_labels))

훈련 데이터의 크기: 885
테스트 데이터의 크기: 222


### 모델 만들기

In [ ]:
# GPU가 잘 사용되는지 테스트할 때는 아래 코드를 사용해 보세요
import torch
torch.rand(10).to("cuda:0")
torch.rand(10).cuda()

tensor([0.5782, 0.0600, 0.2846, 0.2007, 0.5014, 0.3139, 0.4654, 0.1612, 0.1568,
        0.2083], device='cuda:0')

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
num_labels = 14

model = BertForSequenceClassification.from_pretrained("klue/bert-base",
        num_labels=num_labels, problem_type="multi_label_classification")

model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# 옵티마이저 선택
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 에포크 설정
epochs = 15
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))  # hh:mm:ss

In [ ]:
def multi_label_metrics(predictions, labels, threshold=0.5):

    # 모델의 예측에 대해서 시그모이드 함수값을 통과시킨다. (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    print('sigmoid:', sigmoid, ', probs:', probs)

    # 만약 threshold 값을 넘는 경우에는 1로 예측했다고 간주한다.
    # threshold 값은 일반적으로 로지스틱 회귀 방식에 의하여 0.5를 선택하는 것이 일반적이다.
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    y_true = labels

    # 사용 가능한 메트릭들을 사용한다.
    accuracy = accuracy_score(y_true, y_pred)
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')

    # 메트릭 결과에 대해서 리턴
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average,
               'roc_auc': roc_auc}

    return metrics


GPU out of memory 에러 발생시 캐시 지우기

In [ ]:
import torch
torch.cuda.empty_cache()

import gc

gc.collect()

torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 5            |        cudaMalloc retries: 5         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   1299 MiB |  14771 MiB |   6778 GiB |   6777 GiB |\n|       from large pool |   1297 MiB |  14768 MiB |   6756 GiB |   6754 GiB |\n|       from small pool |      1 MiB |      4 MiB |     22 GiB |     22 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   1299 MiB |  14771 MiB |   6778 GiB |   6777 GiB |\n|       from large pool |   1297 MiB |  14768 MiB |

In [ ]:
# 랜덤 시드값.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    t0 = time.time()
    total_loss = 0

    model.train()

    # 에포크 5번마다 모델 저장한다
    if epoch_i % 5 == 0:
        torch.save(model.state_dict(), f"BERT_multilabel_model_{epoch_i}.pt")

    for step, batch in tqdm(enumerate(train_dataloader)):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # gradient clipping if it is over a threshold
        optimizer.step()
        scheduler.step()

        model.zero_grad()



    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

======== Epoch 1 / 15 ========


56it [01:01,  1.09s/it]



  Average training loss: 0.2165
  Training epcoh took: 0:01:02
======== Epoch 2 / 15 ========


56it [01:01,  1.10s/it]



  Average training loss: 0.1904
  Training epcoh took: 0:01:01
======== Epoch 3 / 15 ========


...


  Average training loss: 0.0598
  Training epcoh took: 0:01:02
======== Epoch 14 / 15 ========


56it [01:01,  1.10s/it]



  Average training loss: 0.0587
  Training epcoh took: 0:01:01
======== Epoch 15 / 15 ========


56it [01:01,  1.10s/it]


  Average training loss: 0.0571
  Training epcoh took: 0:01:01


In [ ]:
epoch_i

14

In [ ]:
torch.save(model.state_dict(), f"BERT_multilabel_model_{epoch_i}.pt")

In [ ]:
# 예측하기

t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for step, batch in tqdm(enumerate(test_dataloader)):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
        accum_logits.append(list(b))

    for b in label_ids:
        accum_label_ids.append(list(b))

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)

14it [00:05,  2.67it/s]


In [ ]:
results = multi_label_metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))

sigmoid: Sigmoid() , probs: tensor([[0.8876, 0.7996, 0.0192,  ..., 0.0128, 0.0089, 0.0200],
        [0.1511, 0.9419, 0.0311,  ..., 0.0073, 0.0056, 0.0166],
        [0.1691, 0.6913, 0.1379,  ..., 0.0051, 0.0035, 0.0138],
        ...,
        [0.0912, 0.6537, 0.0835,  ..., 0.0044, 0.0040, 0.0143],
        [0.7413, 0.9706, 0.0247,  ..., 0.0080, 0.0065, 0.0094],
        [0.0295, 0.9758, 0.0327,  ..., 0.0116, 0.0126, 0.0195]])
Accuracy: 0.3108
F1 (Macro) Score: 0.1767
F1 (Micro) Score: 0.5668
F1 (Weighted) Score: 0.5035
ROC-AUC: 0.7210


In [ ]:
results = multi_label_metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))

sigmoid: Sigmoid() , probs: tensor([[0.1970, 0.7628, 0.1588,  ..., 0.0372, 0.0307, 0.0769],
        [0.5800, 0.6887, 0.1566,  ..., 0.0370, 0.0340, 0.0648],
        [0.2225, 0.6106, 0.0970,  ..., 0.0280, 0.0284, 0.0523],
        ...,
        [0.1380, 0.7825, 0.1634,  ..., 0.0408, 0.0359, 0.0785],
        [0.2959, 0.2505, 0.1168,  ..., 0.0342, 0.0334, 0.0454],
        [0.2576, 0.6955, 0.1578,  ..., 0.0307, 0.0285, 0.0741]])
accuracy 구하기 전
y_true.shape: (222, 14)
y_pred.shape: (222, 14)
accuracy: 0.3153153153153153
f1_macro_average: 0.13540670121242182
f1_micro_average: 0.5507246376811594
f1_weighted_average: 0.46064875244405096
roc_auc: 0.7134253831876277
Accuracy: 0.3153
F1 (Macro) Score: 0.1354
F1 (Micro) Score: 0.5507
F1 (Weighted) Score: 0.4606
ROC-AUC: 0.7134


In [ ]:
# 모델 저장
# torch.save(model.state_dict(), path+"BERT_multilabel_model.pt")

In [ ]:
# 모델 로드
# model.load_state_dict(torch.load(path+"BERT_multilabel_model.pt"))

### 예측

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512,
                return_all_scores=True, function_to_apply='sigmoid')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
genre_to_idx

{'댄스': 0,
 '발라드': 1,
 'R&B/Soul': 2,
 '랩/힙합': 3,
 '성인가요/트로트': 4,
 '록/메탈': 5,
 '국내드라마': 6,
 '일렉트로니카': 7,
 '포크/블루스': 8,
 '크로스오버': 9,
 '애시드/퓨전/팝': 10,
 '클래식': 11,
 '재즈': 12,
 '인디음악': 13}

In [ ]:
for idx, val in enumerate(genre_to_idx):
    print(idx, val)

0 댄스
1 발라드
2 R&B/Soul
3 랩/힙합
4 성인가요/트로트
5 록/메탈
6 국내드라마
7 일렉트로니카
8 포크/블루스
9 크로스오버
10 애시드/퓨전/팝
11 클래식
12 재즈
13 인디음악


In [ ]:
label_dict = {f"LABEL_{idx}" : val for idx, val  in enumerate(genre_to_idx)}

In [ ]:
def prediction(text):
    result = pipe(text)
    return [label_dict[res['label']] for res in result[0] if res['score'] > 0.5]

In [ ]:
lyrics = '''
Bring the fire
거침없이 Yeah
기나긴 밤 이겨냈지
선명한 너의 그 상처는 빛났어
Oh
주먹 쥐고 턱을 올려
그래 Ad Astra
아득한 미래도 내 손안에 잡아
Feeling like a rockstar
Lights on us
쏟아지는 갈채속의 Fire
We go
We go
We go hard
Hard like a criminal
Hard like the beat
Hard like ma flow
너도 원했니
밤이 녹을 만큼
뜨겁고 진한
새로운 히트
멈추지
못 했다지?
Hard like a criminal
Hard like the beat
Hard like ma flow
구미 당겼지
Yeah
멋 내 봐 맛을 봐
누가 누가 와
너를 놀래키나
We go hard
걸음은 킹콩
그 놈 참 물건
잠시 한 눈 좀 팔아도
다시 챔피언
왕관은 주인을 되찾아내
세상의 끝까지
Yeah we don’t stop
Oh
시간이 아까워 Tic Toc
서둘러
Sold out
목말랐던 만큼 원하고 원해 봐
Feeling like a rockstar
Lights on us
걸어 나와 네 순간을 마셔
We bring the fire
We go
We go
We go hard
Hard like a criminal
Hard like the beat
Hard like ma flow
너도 원했니
밤이 녹을 만큼
뜨겁고 진한
새로운 히트
멈추지
못 했다지?
Hard like a criminal
Hard like the beat
Hard like ma flow
구미 당겼지
Yeah
멋 내 봐 맛을 봐
누가 누가 와
너를 놀래키나
We go hard
같이 가볼까
되감을 순 없어
지금 이 순간만 기억해 줘
No looking back
넌 알잖아
미쳐 봐 날아 봐
우린 강해
We get you right
KO
We gon’ knock you out
거기 MC count down to...
Zero
Yesir
We go
We go
We go hard
Hard like a criminal
Hard like the beat
Hard like ma flow
너도 원했지
밤이 녹을 만큼
뜨겁고 진한
새로운 히트
멈추지
못 했다지?
Hard like a criminal
Hard like the beat
Hard like ma flow
구미 당겼지
Yeah
멋 내 봐 맛을 봐
누가 누가 와
너를 놀래키나
We go hard
'''

In [ ]:
lyrics = preprocess([lyrics])

['거침없이 기나긴 밤 이겨냈지선명한 너의 그 상처는 빛났어주먹 쥐고 턱을 올려그래  아득한 미래도 내 손안에 잡아     쏟아지는 갈채속의              너도 원했니밤이 녹을 만큼뜨겁고 진한새로운 히트멈추지못 했다지         구미 당겼지멋 내 봐 맛을 봐누가 누가 와너를 놀래키나  걸음은 킹콩그 놈 참 물건잠시 한 눈 좀 팔아도다시 챔피언왕관은 주인을 되찾아내세상의 끝까지   시간이 아까워  서둘러 목말랐던 만큼 원하고 원해 봐     걸어 나와 네 순간을 마셔                너도 원했니밤이 녹을 만큼뜨겁고 진한새로운 히트멈추지못 했다지         구미 당겼지멋 내 봐 맛을 봐누가 누가 와너를 놀래키나  같이 가볼까되감을 순 없어지금 이 순간만 기억해 줘  넌 알잖아미쳐 봐 날아 봐우린 강해       거기                 너도 원했지밤이 녹을 만큼뜨겁고 진한새로운 히트멈추지못 했다지         구미 당겼지멋 내 봐 맛을 봐누가 누가 와너를 놀래키나']

In [ ]:
result = pipe(lyrics)
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'LABEL_0', 'score': 0.9876783490180969}, {'label': 'LABEL_1', 'score': 0.5617058873176575}, {'label': 'LABEL_2', 'score': 0.03496751934289932}, {'label': 'LABEL_3', 'score': 0.10405497997999191}, {'label': 'LABEL_4', 'score': 0.012061669491231441}, {'label': 'LABEL_5', 'score': 0.0755143016576767}, {'label': 'LABEL_6', 'score': 0.01566191390156746}, {'label': 'LABEL_7', 'score': 0.036784231662750244}, {'label': 'LABEL_8', 'score': 0.02031835727393627}, {'label': 'LABEL_9', 'score': 0.013626680709421635}, {'label': 'LABEL_10', 'score': 0.015070327557623386}, {'label': 'LABEL_11', 'score': 0.014247684739530087}, {'label': 'LABEL_12', 'score': 0.012642097659409046}, {'label': 'LABEL_13', 'score': 0.01169545017182827}]]


In [ ]:
prediction(lyrics)

['댄스', '발라드']